### **L4 기반(Google pro)**
- **불용어후보(7.2, 월)바탕으로 LDA**

### **한글깨짐 방지 아래 코드**

In [1]:
%%capture
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
import matplotlib.font_manager as fm

# 설치된 폰트 목록 출력
for font in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
    if 'Nanum' in font:
        print(font)


/usr/share/fonts/truetype/nanum/NanumSquareB.ttf
/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf
/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf
/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf
/usr/share/fonts/truetype/nanum/NanumGothic.ttf
/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf
/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf
/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf
/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf
/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf
/usr/share/fonts/truetype/nanum/NanumSquareR.ttf
/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf


In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# NanumGothic 폰트를 직접 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family=font_prop.get_name())

plt.rcParams['axes.unicode_minus'] = False
'''
# 테스트 그래프 그리기
plt.plot([1, 2, 3], [4, 5, 6])
plt.title('테스트 그래프', fontproperties=font_prop)
plt.xlabel('X 축', fontproperties=font_prop)
plt.ylabel('Y 축', fontproperties=font_prop)
plt.show()
'''

"\n# 테스트 그래프 그리기\nplt.plot([1, 2, 3], [4, 5, 6])\nplt.title('테스트 그래프', fontproperties=font_prop)\nplt.xlabel('X 축', fontproperties=font_prop)\nplt.ylabel('Y 축', fontproperties=font_prop)\nplt.show()\n"

### **재시작**

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import time
import gzip
import pickle
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **파일 읽기**

In [4]:
df= pd.read_pickle('/content/drive/MyDrive/이대과제/data/df_gzip.pkl', compression='gzip')
print(df.shape)
df.head()

(5641608, 10)


,일자,언론사,제목,통합 분류1,통합 분류2,통합 분류3,기관,특성추출(가중치순 상위 50개),년도,월
0,2010-01-01,세계일보,2010 신년특집-업계 신년 구상/IT/SK 3조 투자 이종산업과 접목 주력,경제>산업_기업,IT_과학>모바일,IT_과학>인터넷_SNS,"FMC,IPE,KT,KTF,SK텔레콤,네이버,대상,정보통신","네트워크,3조,네이버,sk텔레콤,일본,사용자,kt,장악력,생산성,융합사업,이종산업,...",2010,1
1,2010-01-01,전자신문,[2010 신년특집] 해외 IT융합 프로젝트-일본 및 아시아국가,경제>자원,NaN,NaN,"민주당,일본,이화학연구소,문부과학성,건설공사,인도네시아,싱가포르,태양광발전소,그린I...","일본,싱가포르,슈퍼컴퓨터,전자폐기물,스마트그리드,대만,전력망,신재생,연구소,각국,차...",2010,1
2,2010-01-01,내일신문,소비양극화 백화점 인터넷쇼핑 성장,경제>산업_기업,경제>유통,경제>자동차,"후레쉬푸드,아울렛,타임스퀘어,슈퍼","백화점,편의점,온라인,대형마트,경쟁력,유통산업발전법,복합몰,유통업,신세계,쇼핑몰,신장세",2010,1
3,2010-01-01,이데일리,2010년 `보건의료분야` 이렇게 달라져요,사회>의료_건강,사회>장애인,NaN,"문화재단,시군구청,정부,사회복,식약청,사회복지,사회복지통합관리망,요양병원","장애인,진료비,의료기관,진단서,이데일리,소비자,건강보험,영유아,치료제,보건소,만4세...",2010,1
4,2010-01-01,이데일리,"이 대통령 ""상반기까진 출구전략 신중""",정치>청와대,정치>외교,NaN,"국무회의,정부,국무위","이데일리,상반기,일자리,선제적,공직자들,공직자,민간기업,해외진출,출구전략,재정지출,...",2010,1


In [6]:
# 필요한 컬럼만 추출
df= df[['일자', '특성추출(가중치순 상위 50개)']]

In [8]:
import time
print('sf')

sf


## **Sampling**

In [9]:
df= df.sample(frac=0.4)

### **불용어**

In [10]:
data = pd.read_excel('/content/drive/MyDrive/이대과제/data/불용어후보.xlsx')
print(data.shape)
data.tail()

# 불용어 처리
stop_words = data[data['불용어_0703'] == 1]['단어'].tolist()

(4878, 9)


,단어,불룡어?,유형,이대,불용어_0622,불용어_0625,불용어_0627,불용어_0702,불용어_0703
4873,전년도,NaN,추가작업_0703,NaN,NaN,NaN,NaN,NaN,1.0
4874,전국,NaN,추가작업_0703,NaN,NaN,NaN,NaN,NaN,1.0
4875,개최,NaN,추가작업_0703,NaN,NaN,NaN,NaN,NaN,1.0
4876,우수,NaN,추가작업_0703,NaN,NaN,NaN,NaN,NaN,1.0
4877,전국적,NaN,추가작업_0703,NaN,NaN,NaN,NaN,NaN,1.0


### **CountVectorizer, TfidfVectorizer,TfidfTransformer**

## 1. 사이킷런을 이용한 토픽 모델링
- **[파이썬 텍스트 마이닝 완벽 가이드](https://github.com/wikibook/textmining/blob/main/Ch%2007.%20%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb)**
### **LDA**

## **<font color='blue'> gensim을 이용하여 LDA**

- 하이퍼파라미터 3개 부터 10개 LDA수행 후 토픽 개수 결정
- topic : 당신이 가설로 잡은 토픽의 갯수는?
- chunksize : 얼마나 많은 문서가 훈련 알고리즘에 사용되는가?
- 만약에 빠른 학습이 중요하시다면, 청크사이즈를 키워서 돌려봅시다!
- Hoffman의 논문에 의하면 Chunksize는 모델 품질에 영향을 미치지만 차이그 그렇게 크진 않다고 합니다!
- passes : 패스는 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도를 제어한다고 합니다. epochs 와 같은 용어 같다! model를 학습시키는 횟수를 말하는것 같아요! model sampling?
- iteration : 각각 문서에 대해서 루프를 얼마나 돌리는지를 제어한다고 합니다.
pass & iteration 은 최대한 많은게 좋다!
- eval_every = 1 in LdaModel
- alpha, eta = auto, 디리클레 분포의 감마함수에 대한 파라미터입니다!

In [11]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
#from gensim.utils import lemmatize, simple_preprocess # lemmatize is no longer in gensim.utils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import RegexpTokenizer

### **정규표현식으로 필요한 단어만 추출하거나, 불필요한 단어를 삭제**

In [12]:
RegTok = RegexpTokenizer("[\w']{2,}") # 정규포현식으로 토크나이저를 정의
food_stops = set(stop_words) # 불용어를 가져옴

# Compile the additional regular expressions
pattern_mixed = re.compile(r'(?=.*\d)(?=.*[a-zA-Z가-힣])') # 숫자와 알파벳 또는 한글이 모두 포함된 문자열을 찾는 정규 표현식
pattern_korean_english = re.compile(r'^[가-힣a-zA-Z]+$')  # 한글과 알파벳으로만 구성된 문자열을 찾는 정규 표현식

# Define the pattern to remove words containing both numbers and either English or Korean characters
pattern_remove = re.compile(r'\d+[a-zA-Z가-힣]+|[a-zA-Z가-힣]+\d+')  # 90ml', '1항', '3qo', '30개국' 등과 같이 수자와 영문, 수자와 한글이 나오는 단어를 제거

def tokenizer(text):
    tokens = RegTok.tokenize(text)
    # Filter out stop words and apply the additional patterns
    words = [word for word in tokens if (word not in food_stops) and
             (len(word) >= 2) and
             not pattern_remove.match(word) and
             (pattern_mixed.match(word) or pattern_korean_english.match(word))]
    return words

In [13]:
# Check the data types in your DataFrame
print(df['특성추출(가중치순 상위 50개)'].dtypes)

# If the column is not of type string, convert it:
df['특성추출(가중치순 상위 50개)'] = df['특성추출(가중치순 상위 50개)'].astype(str)

texts = [tokenizer(news) for news in df['특성추출(가중치순 상위 50개)']]

object


### **기본 현황**

In [14]:
from tqdm import tqdm
# 토큰화 결과로부터 dictionay 생성
dictionary = Dictionary(texts)
print('#Number of initial unique words in documents:', len(dictionary))

# 문서 빈도수가 너무 적거나 높은 단어를 필터링하고 특성을 단어의 빈도 순으로 선택
dictionary.filter_extremes(keep_n=2000, no_below=5, no_above=0.5)
print('#Number of unique words after removing rare and common words:', len(dictionary))

# 카운트 벡터로 변환
corpus = [dictionary.doc2bow(text) for text in tqdm(texts)]

# Create Corpus: Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in data_words if text not in stop_words]
print('#Number of unique tokens: %d' % len(dictionary))
print('#Number of documents: %d' % len(corpus))

#Number of initial unique words in documents: 1870409
#Number of unique words after removing rare and common words: 2000


100%|██████████| 2256643/2256643 [00:19<00:00, 115054.26it/s]

#Number of unique tokens: 2000
#Number of documents: 2256643


### **Topic수에 따라 Clustering하고 시각화**

In [15]:
%%capture
!pip install pyLDAvis

### **혼란도와 토픽 응집도를 이용한 최적값 선택**
- https://github.com/wikibook/textmining/blob/main/Ch%2007.%20%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb

## **전체 코드 : Topic의 개수를 정하고 파일로 저장**
#### **Topic개수 및 기간을 정할 수 있음**

In [ ]:
from gensim.models import LdaModel
import pyLDAvis.gensim # Now this import should work , !pip install pyLDAvis
from tqdm import tqdm
from datetime import date

#  Topics 입력 ###########################################################>>>>>>>
topics_list = [6,7]

for num_topics in  tqdm(topics_list):
    passes = 5
    %time model = LdaModel(corpus=corpus, id2word=dictionary,\
                           passes=passes, num_topics=num_topics, \
                           random_state=7)

    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)

    pyLDAvis.save_html(vis, str(num_topics) + '_topics_' + str(date.today()) + '---' + '.html')

    # Topic 선형관계식 : 주제 모델링 결과 저장 예제
    topics = model.print_topics()
    # 텍스트 파일로 저장
    with open(f'{str(num_topics)}_topics.txt', 'w', encoding='utf-8') as file:
        for topic in topics:
            file.write(str(topic) + '\n')

    ### **1. 문서집단(corpus)을 바탕으로 온라인 기사별로 토픽별 비중을 확인**
    ###################################################################
    doc_topics =[]
    for topic_list in tqdm(model[corpus]):
        ulist =[]
        for j in range(len(topic_list)):
            ulist.append(topic_list[j][1])
        doc_topics.append(ulist)

    pd.DataFrame(doc_topics).head(3)

    ###################   LDA 년도별 문서 분포 결과 출력       ####################
    topic_list = np.argmax(pd.DataFrame(doc_topics).values, axis=1).tolist()

    # Sample data (for demonstration purposes, replace with your actual data)
    tf = pd.DataFrame({
        '일자': pd.date_range(start='2010-01-01', end='2024-12-31', freq='M'),
        '문서': np.random.randint(0, 100, size=(180,))
    })

    # Ensure topic_list has the same length as tf########################
    # Assuming topics####################################################
    topic_num = num_topics
    #####################################################################
    doc_topics = np.random.rand(len(tf), topic_num)
    topic_list = np.argmax(doc_topics, axis=1).tolist()

    # LDA 년도별 문서 분포 결과 출력
    start_year = 2010
    end_year = 2024
    result_df = pd.DataFrame()  # Initialize result_df before the loop

    for year in tqdm(range(start_year, end_year + 1)):
        start_date = str(year) + "-01-01"
        end_date = str(year) + "-12-31"
        dt_idx = (tf['일자'] >= start_date) & (tf['일자'] <= end_date)
        sub_tf = tf[dt_idx]

        # Ensure indices are within bounds
        valid_indices = [i for i in dt_idx.index[dt_idx] if i < len(topic_list)]
        sub_topic_list = [topic_list[i] for i in valid_indices]

        result = []
        for t in range(max(topic_list) + 1):
            # Count occurrences of each topic in sub_topic_list
            num = sub_topic_list.count(t)
            result.append(num)
        result = pd.Series(result, name=str(year))
        result_df = pd.concat([result_df, result], axis=1)

    time.sleep(10)

    # 비율로 만들기
    result_df = result_df.T.apply(lambda x: x / x.sum(), axis=1)
    result_df.head(3)

    ######################## 년도별 비중 ############
    ax = result_df.plot.barh(stacked=True,legend=False,figsize=(25, 10), title="Topic Proportion(%)")
    ax.legend(bbox_to_anchor=(1.03, 1),loc=2, borderaxespad=0.)

    for p in ax.patches:
        left, bottom, width, height = p.get_bbox().bounds
        ax.annotate("%.1f"%(width*100), xy=(left+width/2, bottom+height/2), ha='center', va='center')

    # Save the figure as a PNG file
    ax.figure.savefig(str(num_topics) + '_topics_비중.png')
    time.sleep(10)

    ############  년도별 비중 추세 ###############
    # 카운트 벡터로 변환
    corpus = [dictionary.doc2bow(text) for text in texts]

    doc_topics =[]
    for topic_list in model[corpus]:
        ulist =[]
        for j in range(len(topic_list)):
            ulist.append(topic_list[j][1])
        doc_topics.append(ulist)

    time.sleep(10)
    pd.DataFrame(doc_topics).head(3)

    trend_data = pd.DataFrame(doc_topics)
    trend_data['start'] = df['일자'].values
    trend_data = trend_data.set_index('start')
    trend_data = trend_data.resample('M').mean()
    trend_data.head(3)

    fig, axes = plt.subplots(num_topics, 1, sharex='col', figsize=(25, 20))

    for col, ax in zip(trend_data.columns.tolist(), axes.ravel()):
        ax.set_title(f'Topic {col}: 월별 추세선', fontsize=18)
        ax.axes.xaxis.set_visible(False)
        ax.plot(trend_data[col])
    ax.figure.savefig(f'{num_topics}_topic의 비중 추세.png')


    time.sleep(10)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 14min 38s, sys: 3.71 s, total: 14min 41s
Wall time: 14min 39s


153

150

149

136

145

147


 11%|█         | 252603/2256643 [00:28<03:40, 9091.28it/s]

## **END**